In [32]:
import glob
import os
import fuse, causal_utils
glob.glob("/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/*/")

['/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/outputs_bert_base_overlapping_korn_ps3class_3/',
 '/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/outputs_bert_base_overlapping_korn_ps3class_2/',
 '/ist/ist-share/scads/can/robust_nlu/korn_reweight/nli/outputs_bert_base_overlapping_korn_ps3class_1/']

In [97]:
!ls /raid/can/nli_models/baseline/outputs_bert_base_clark_1_seed13370

outputs_bert_base_clark_1_seed13370  outputs_bert_base_clark_1_seed33370
outputs_bert_base_clark_1_seed23370


In [100]:
import pandas as pd
import numpy as np
from scipy.special import softmax
from scipy.special import expit

model_path='/raid/can/nli_models/baseline/outputs_bert_base_clark_1_seed13370/'
data_path='/ist/users/canu/debias_nlu/data/nli/'
df = pd.read_json(model_path+'raw_train.jsonl', lines=True)
fusion = fuse.sum_fuse

# Get avg  prob of bert

In [101]:
list_probs = []
for i in df['probs']:
    list_probs.extend(i)
x=np.array(list_probs)
avg=np.average(x,axis=0)

In [102]:
data_path

'/ist/users/canu/debias_nlu/data/nli/'

In [103]:
!ls /ist/users/canu/debias_nlu/data/nli/

check_distill.jsonl
dev_mismatched_hard.jsonl
dev_prob_korn_full_lr_overlapping_sample_weight_3class.jsonl
dev_prob_korn_lr_overlapping_sample_weight_3class.jsonl
gam_sample_weight_clark.jsonl
gam_sample_weight_focal.jsonl
gam_sample_weight_ps.jsonl
hans_prob_korn_full_lr_overlapping_sample_weight_3class.jsonl
hans_prob_korn_lr_overlapping_sample_weight_3class.jsonl
heuristics_evaluation_set.jsonl
heuristics_evaluation_set_sample_weight.jsonl
heuristics_evaluation_set.txt
korn_full_lr_sample_weight.jsonl
multinli_1.0_dev_matched.jsonl
multinli_1.0_dev_matched_overlap.jsonl
multinli_1.0_dev_matched_weighted.jsonl
multinli_1.0_dev_mismatched.jsonl
multinli_1.0_dev_mismatched_overlap.jsonl
multinli_1.0_train.jsonl
multinli_1.0_train_small.jsonl
naik_stress_test
newfile.txt
README.md
small_utama_github_poe.jsonl
snli_1.0_test_hard.jsonl
test_distill.jsonl
test_kaushik_combined.jsonl
test_kaushik_RH.jsonl
test_kaushik_RP.jsonl
test_prob_korn_full_lr_overlapping_sample_weight_3class.jsonl
te

#  get score from the bias model

In [104]:
df_dev = pd.read_json(data_path+'dev_prob_korn_lr_overlapping_sample_weight_3class.jsonl', lines=True)

In [105]:
dev_score=[b for b in df_dev['bias_probs'] ]

In [106]:
dev_score=np.array(dev_score)

In [107]:
dev_score

array([[0.17841856, 0.36916586, 0.45241557],
       [0.16769396, 0.36900166, 0.46330438],
       [0.30998087, 0.35088779, 0.33913134],
       ...,
       [0.43164769, 0.31369753, 0.25465478],
       [0.43164769, 0.31369753, 0.25465478],
       [0.19428596, 0.3688092 , 0.43690484]])

In [108]:
# bias_score=softmax(np.log(avg+dev_score))
# bias_score=softmax(np.log(avg)+np.log(dev_score))
# bias_score=np.log(expit(avg+dev_score))
bias_score=fusion(avg,dev_score)

In [109]:
bias_score

array([[-0.47041369, -0.40222965, -0.37502951],
       [-0.47445166, -0.40228403, -0.37163697],
       [-0.42304967, -0.40831992, -0.4118552 ],
       ...,
       [-0.38273091, -0.4209421 , -0.44117782],
       [-0.38273091, -0.4209421 , -0.44117782],
       [-0.46448882, -0.40234778, -0.3799061 ]])

In [110]:
import pandas as pd
df_gt = pd.read_json(data_path+'multinli_1.0_dev_matched.jsonl', lines=True)
gt_labels=list(df_gt['gold_label'])

In [111]:
result_path=model_path
df_bert = pd.read_json(model_path+'raw_m.jsonl', lines=True)

In [112]:
# ent = []
PoE_prob = []
for p,h in zip(df_bert['probs'],dev_score):
#     new_poe = softmax(np.log(np.array(p))+np.log(dev_score))
    new_poe = fusion(np.array(p),h)
#     ent.append(entropy(p, base=3))
    PoE_prob.append(new_poe)

In [113]:
debias_scores = []
for p,b in zip(PoE_prob,bias_score):
#     e=entropy(p, base=3)
    debias_scores.append(p-b)

In [114]:
key = {0:"entailment",1:"contradiction",2:"neutral"}
labels = []
for i in debias_scores:
    labels.append(key[np.argmax(i)])
df_bert['debias_label']=labels

In [115]:
from scipy.stats import entropy

In [116]:
entropy([1/3, 1/3,1/3], base=3)

0.9999999999999998

In [117]:
def get_ans(ans):
    if ans == 0:
        return 'entailment'
    elif ans==1:
        return 'contradiction'
    else:
        return 'neutral'
    
# y0m0
import pickle
modelname='mnli_lr_model.sav'
loaded_model = pickle.load(open(modelname, 'rb'))
x0=loaded_model.predict_proba(np.array([[0,0,0.41997876976119086]]))
m0=avg
y0m0=fusion(x0,m0)    

In [118]:
factual = []
counter = []
all_nie = []
all_nde = []
for i in range(len(gt_labels)): 
    y1m1 = PoE_prob[i]
    y1m0 = bias_score[i]
    TE = y1m1 - y0m0
    NDE = bias_score[i] - y0m0
    y0m1= fusion(x0,np.array(df_bert['probs'][i]) )
    NIE = y1m1 - y1m0
    factual_ans = np.argmax(df_bert['probs'][i])
    factual_ans = get_ans(factual_ans)
    factual_correct = factual_ans==gt_labels[i]   
    factual.append(factual_correct)
    all_nde.append(NDE[0][0])
    all_nie.append(NIE[0])
    if  (NIE[0]/TE[0][0])>0.1:
        cf_ans = np.argmax(np.array(df_bert['probs'][i]-dev_score[i]))
        cf_ans = get_ans(cf_ans)  
        cf_correct = cf_ans==gt_labels[i]
    else:
#         print(cf_ans)
        cf_correct = factual_ans ==gt_labels[i]
    counter.append(cf_correct)
    
#     np.array(df_bert['probs'][i]-bias_score)
#     gt_labels[i]
print(np.array(all_nie).mean())
print(np.array(all_nde).mean())

-0.02788629555058882
0.0008568151657749869


In [127]:
print(sum(factual)/10000,sum(counter)/10000)

0.8276 0.8267


In [128]:
TE = PoE_prob[1] - y0m0

In [129]:
labels[i]

'contradiction'

In [130]:
-0.487301456504589

PoE_prob[0]

array([-0.60723412, -0.52248184, -0.21195785])

In [131]:
np.array(df_bert['probs'][i]-bias_score)

array([[0.47065834, 1.40113924, 0.37587522],
       [0.4746963 , 1.40119362, 0.37248268],
       [0.42329432, 1.40722952, 0.4127009 ],
       ...,
       [0.38297556, 1.41985169, 0.44202353],
       [0.38297556, 1.41985169, 0.44202353],
       [0.46473347, 1.40125737, 0.38075181]])

In [132]:
labels[5000]

'contradiction'

In [133]:
np.array(df_bert['probs'][i])

array([2.44645227e-04, 9.98909593e-01, 8.45707196e-04])

In [134]:
np.argmax(df_bert['probs'][i])

1

In [259]:
#PM # 0.032416620996870094 # utama best seed  0.04260506151060745
#PM # 0.0879242620437139 # PoE 0.08649503037239671
#PM # 0.10864416858494 # Baseline 0.10295758381662952